In [ ]:
! pip install "whisper-at@git+https://github.com/Zivkr/whisper-at.git#subdirectory=package/whisper-at"

In [1]:
import torch
import torch.nn as nn
import whisper_at as whisper
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# download a sample audio
# !pip -q install wget
# import wget,IPython
# wget.download('https://www.dropbox.com/s/7eznyazmc1pmw9h/case_closed.wav?dl=1', '/home/zivk/Documents/Deep Learning/whisper-at/sample_audio.flac')
# IPython.display.Audio('/home/zivk/Documents/Deep Learning/whisper-at/sample_audio.flac')

In [2]:
audio_tagging_time_resolution = 10
model = whisper.load_model("tiny")

# Freeze all layers except the last one
for param in model.parameters():
    param.requires_grad = False
    
model.at_model.mlp_layer = nn.Identity()

print(model)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-3): 4 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=False)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (out): Linear(in_features=384, out_features=384, bias=True)
        )
        (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
        (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((384,), eps=1e-05,

In [3]:
last_layer_size = model.at_model.layer_tr.mlp_ln.weight.shape[0]
result = model.transcribe("/home/zivk/Documents/Deep Learning/Datasets/ESC-50-master/audio/1-137-A-32.wav", at_time_res=audio_tagging_time_resolution, mlp_ln_output=last_layer_size)
# ASR Results
# print(result["text"])
print(result["audio_tag"].shape)

/home/zivk/Documents/Deep Learning/whisper-at/venv/lib/python3.10/site-packages/whisper_at/transcribe.py:121: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


torch.Size([1, 384])


In [15]:
train_features = result["audio_tag"]

train_labels = torch.tensor([32])

classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, multi_class="ovr", verbose=1)

classifier.fit(train_features, train_labels)
# Evaluate using the logistic regression classifier
# predictions = classifier.predict( val_features)

# accuracy = accuracy_score(val_labels, predictions)

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 32

In [7]:
train_labels = torch.tensor([32])

print(train_labels.shape)

torch.Size([1])
